In [1]:
"""import statements as required"""
import pandas as pd

In [7]:
"""Multiple dataframes based on input files"""
filenames = ["results", "goalscorers", "shootouts"]
dataframes = {}
for filename in filenames:
    df = pd.read_csv(f"Data/Original/{filename}.csv")
    dataframes[filename] = df

print("Original match count", len(dataframes["results"]))

Original match count 47980


In [ ]:
"""Combine the results df with the goalscoreres df and save to csv for future use"""
goals = dataframes["goalscorers"]
shootouts = dataframes["shootouts"]
results = dataframes["results"].iloc[10000 * i:]
combined = pd.DataFrame(columns = ["date", "home_team", "away_team", "home_score", "away_score", "home_team_victory", "draw", "tournament_name", "city", "country", "neutral",
            "scoring_team", "scoring_player", "minute", "own_goal", "penalty", "shootout_required", "first_shooter", "winner"])

for result_index, result_row in results.iterrows():

    date = result_row["date"]
    home_team = result_row["home_team"]
    away_team = result_row["away_team"]
    home_score = result_row["home_score"]
    away_score =  result_row["away_score"]
    home_team_victory = home_score > away_score
    draw = home_score == away_score
    tournament_name = result_row["tournament"]
    city = result_row["city"]
    country = result_row["country"]
    neutral = result_row["neutral"]

    df = goals.loc[goals["date"] == date]
    df = df.loc[df["home_team"] == home_team]
    df = df.loc[df["away_team"] == away_team]

    if len(df):
        for goal_index, goal_row in df.iterrows():
            scoring_team = goal_row["team"]
            scoring_player = goal_row["scorer"]
            minute = goal_row["minute"]
            own_goal = goal_row["own_goal"]
            penalty = goal_row["penalty"]
            combined.loc[len(combined)] = [date, home_team, away_team, home_score, away_score, home_team_victory, draw, tournament_name, city, country, neutral,
                    scoring_team, scoring_player, minute, own_goal, penalty]
    else:
        scoring_team = None
        scoring_player = None
        minute = None
        own_goal = None
        penalty = None
        combined.loc[len(combined)] = [date, home_team, away_team, home_score, away_score, home_team_victory, draw, tournament_name, city, country, neutral,
            scoring_team, scoring_player, minute, own_goal, penalty]

combined.to_csv("Data/Interm/combined.csv", index=False)

In [35]:
"""Add more columns to the combined CSV accounting for shootout datapoints"""
combined = pd.read_csv("combined.csv")
shootouts = dataframes["shootouts"]

everything = pd.DataFrame(columns = ["date", "home_team", "away_team", "home_score", "away_score", "home_team_victory", "draw", "tournament_name", "city", "country", "neutral",
            "scoring_team", "scoring_player", "minute", "own_goal", "penalty", "shootout_required", "first_shooter", "winner"])
for result_index, result_row in combined.iterrows():
    date = result_row["date"]
    home_team = result_row["home_team"]
    away_team = result_row["away_team"]
    home_score = result_row["home_score"]
    away_score =  result_row["away_score"]
    home_team_victory = result_row["home_team_victory"]
    draw = home_score == result_row["away_score"]
    tournament_name = result_row["tournament_name"]
    city = result_row["city"]
    country = result_row["country"]
    neutral = result_row["neutral"]
    scoring_team = result_row["scoring_team"]
    scoring_player = result_row["scoring_player"]
    minute = result_row["minute"]
    own_goal = result_row["own_goal"]
    penalty = result_row["penalty"]

    df = shootouts.loc[shootouts["date"] == date]
    df = df.loc[df["home_team"] == home_team]
    df = df.loc[df["away_team"] == away_team]

    if len(df):
        for shootout_index, shootout_row in df.iterrows():
            shootout_required = True
            first_shooter = shootout_row["first_shooter"]
            winner = shootout_row["winner"]
            everything.loc[len(everything)] = [date, home_team, away_team, home_score, away_score, home_team_victory, draw, tournament_name, city, country, neutral,
                    scoring_team, scoring_player, minute, own_goal, penalty, shootout_required, first_shooter, winner]
    else:
        shootout_required = False
        first_shooter = None
        winner = None
        everything.loc[len(everything)] = [date, home_team, away_team, home_score, away_score, home_team_victory, draw, tournament_name, city, country, neutral,
            scoring_team, scoring_player, minute, own_goal, penalty, shootout_required, first_shooter, winner]
        
everything.to_csv(f"Data/Interm/everything.csv", index=False)

In [7]:
"""Adjust data to account for shootouts"""
combined = pd.read_csv("Interm/everything.csv")
combined.loc[combined["home_team"] == combined["winner"], "home_team_victory"] = True
combined.loc[combined["shootout_required"] == True, "draw"] = False

combined.to_csv("Interm/combined.csv", index=False)

In [8]:
"""Break data into training data and test data"""
training_data = combined.loc[combined["tournament_name"] == "FIFA World Cup qualification"]
test_data = combined.loc[combined["tournament_name"] == "FIFA World Cup"]

training_len = len(training_data)
test_len = len(test_data)

print("FIFA qualification match count:", training_len)
print("FIFA Cup match count:", test_len)
print("Test data percentage:", round(100 * test_len / (test_len + training_len), 2))

FIFA qualification match count: 23719
FIFA Cup match count: 2798
Test data percentage: 10.55


In [9]:
"""Save test and training datasets"""
training_data.to_csv("Working/training.csv", index=False)
test_data.to_csv("Working/test.csv", index=False)
print("done")

done
